In [ ]:
%pip install openai
%pip install python-dotenv
%pip install scipy
%pip install tenacity
%pip install tiktoken
%pip install termcolor 

In [1]:
import os
import openai
import json
from dotenv import load_dotenv
load_dotenv()
from openai import OpenAI
openai.api_key = os.getenv("OPENAI_API_KEY")
import json
from tenacity import retry, wait_random_exponential, stop_after_attempt
from termcolor import colored  
from enum import Enum

GPT_MODEL = "gpt-3.5-turbo-0613"
client = OpenAI()

In [3]:
@retry(wait=wait_random_exponential(multiplier=1, max=40), stop=stop_after_attempt(3))
def chat_completion_request(messages, tools=None, tool_choice=None, model=GPT_MODEL):
    try:
        response = client.chat.completions.create(
            model=model,
            messages=messages,
            tools=tools,
            tool_choice=tool_choice,
        )
        return response
    except Exception as e:
        print("Unable to generate ChatCompletion response")
        print(f"Exception: {e}")
        return e


In [4]:
def pretty_print_conversation(messages):
    role_to_color = {
        "system": "red",
        "user": "green",
        "assistant": "blue",
        "function": "magenta",
    }
    
    for message in messages:
        if message["role"] == "system":
            print(colored(f"system: {message['content']}\n", role_to_color[message["role"]]))
        elif message["role"] == "user":
            print(colored(f"user: {message['content']}\n", role_to_color[message["role"]]))
        elif message["role"] == "assistant" and message.get("function_call"):
            print(colored(f"assistant: {message['function_call']}\n", role_to_color[message["role"]]))
        elif message["role"] == "assistant" and not message.get("function_call"):
            print(colored(f"assistant: {message['content']}\n", role_to_color[message["role"]]))
        elif message["role"] == "function":
            print(colored(f"function ({message['name']}): {message['content']}\n", role_to_color[message["role"]]))

In [41]:
# --------------------------------------------------------------
# Use OpenAI’s Function Calling Feature
# --------------------------------------------------------------

# Open the JSON file
with open('course_subjects.json', 'r') as file:
    # Load the JSON data into a Python list
    subjects = json.load(file)
    
tools = [
    # {
    #     "type": "function",
    #     "function": {
    #     "name": "filter_by_rating",
    #     "description": "Get a class by its rating (0 - 5). If a number is not provided, interpret their opinion to fit the range (0-5).", 
    #     "parameters": {
    #         "type": "object",
    #         "properties": {
    #             "comparison_operator": {
    #                 "type": "string",
    #                 "enum": ["$lt", "$gt", "$gte", "$lte"],
    #                 "description": "A comparison operator",
    #             },
    #             "rating": {
    #                 "type": "number",
    #                 "description": "The rating (a number) for the class",
    #             },
    #         },
    #         "required": ["comparison_operator", "rating"],
    #     },
    #     }
    # },
    
    {
        "type": "function",
        "function": {
        "name": "CourseFilter",
        "description": "Provide filters for a course based on conditions.", 
        "parameters": {
            "type": "object",
            "properties": {
                "subject_code": {
                    "type": "string",
                    "enum": [str(key) for key in subjects.keys()],
                    "description": "A code for the subject of instruction",
                },
                "rating": {
                    "type": "number",
                    "description": "The rating (a number with one significant digit) for the class (0 - 4). If a number is not provided, interpret the given opinion to fit the range. A good, or average, class should be 3.5",
                },
                "comparison_operator_rating": {
                    "type": "string",
                    "enum": ["$lt", "$gt", "$gte", "$lte"],
                    "description": "A comparison operator for the class rating",
                },
                "workload": {
                    "type": "number",
                    "description": "The workload (a number with one significant digit) for the class (0 - 4). If a number is not provided, interpret the given opinion to fit the range.",
                },
                "comparison_operator_workload": {
                    "type": "string",
                    "enum": ["$lt", "$gt", "$gte", "$lte"],
                    "description": "A comparison operator for the class workload",
                },
                "comparison_operator_workload": {
                    "type": "string",
                    "enum": ["$lt", "$gt", "$gte", "$lte"],
                    "description": "A comparison operator for the class workload",
                },
                
            },
            "required": ["comparison_operator", "rating"],
        },
        }
    }
]

In [42]:
messages = []
messages.append({"role": "system", "content": "Don't make assumptions about what values to plug into functions. Ask for clarification if a user request is ambiguous."})
messages.append({"role": "user", "content": "Give me a good and easy CS class"})
chat_response = chat_completion_request(
    messages, tools=tools
)
assistant_message = chat_response.choices[0].message
messages.append(assistant_message)
assistant_message.tool_calls[0].function.arguments


Unable to generate ChatCompletion response
Exception: Error code: 400 - {'error': {'message': "Invalid schema for function 'CourseFilter': 'ACCT,USAF,AFAM,AFST,AKKD,ASL,AMST,GREK,ANTH,AMTH,APHY,ARBC,ARCG,ARCH,ARMN,ART,ASTR,BNGL,B&BS,BIOL,BENG,BIS,BRST,BURM,C&MP,CBIO,CENG,CHEM,CHER,CHLD,CHNS,CDE,CLCV,CLSS,MEDR,CGSC,CSYC,CSBF,CSBK,CSBR,CSDC,CSES,CSGH,CSJE,CSMC,CSMY,CSPC,CSSY,CSSM,CSTD,CSTC,CB&B,CPLT,CPSC,CSEC,CPAR,MEDC,CZEC,DEVN,DRST,DIR,DISR,DRAM,DRMA,DUTC,EMST,EPS,EALL,EAST,E&EB,ECON,EDST,EGYP,EENG,ENRG,ENAS,ENGL,ELP,ENV,ENVE,EHS,EVST,EPH,EMD,EP&E,ER&M,E&RS,EXCH,EXPA,FILM,FNSH,F&ES,FREN,GENE,G&G,GMAN,GLBL,HPM,HLTH,HEBR,HNDI,HSHM,HIST,HSAR,HMRT,HUMS,HGRN,IBIO,IDRS,INDN,INP,IMED,ITAL,JAPN,JDST,KHMR,SWAH,KREN,LATN,LAST,LAW,LING,LITR,MGT,MGMT,MRES,MHHR,MATH,MENG,MDVL,MESO,MBIO,MGRK,MMES,MTBT,MB&B,MCDB,MUSI,NAVY,NELC,NPLI,NSCI,NURS,OTTM,PATH,PERS,PHAR,PHIL,OLPA,PA,PHYS,PLSH,PLSC,PORT,PRAC,CAND,QUAL,PSYC,PHUM,PNJB,REL,RLST,RNST,ROMN,RUSS,RSEE,SKRT,MD,MUS,SCIE,SMTC,SBCR,SNHL,SLAV,SBS,SOCY,SAS

AttributeError: 'BadRequestError' object has no attribute 'choices'

In [32]:
print(assistant_message.tool_calls[0].function.arguments)

{
  "subject_code": "CPSC",
  "rating": 4,
  "comparison_operator_rating": "$gte",
  "workload": 2,
  "comparison_operator_workload": "$lte"
}
